# DM - PROJECT
## Game of Thrones first and third book analysis
---------

I am boos


## Data Cleaning


### Imports

In [54]:
from siuba import filter
import siuba
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from tidytext import unnest_tokens
nltk.download('punkt')
nltk.download('stopwords')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\drago\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\drago\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [63]:
game_before_df = pd.read_table("../resources/AGameOfThrones.txt", encoding="UTF-8")
swords_before_df = pd.read_table("../resources/StormOfSwords.txt", encoding="UTF-8")

In [64]:
def clean_dataframe(before_df) -> pd.DataFrame:
    df = before_df.copy()
    df.set_axis(
        ['content'], axis=1, inplace=True
    )

    df = (unnest_tokens(df, "word", "content"))
    df.reset_index(drop=True, inplace=True)
    df = df[df.word.notnull()].reset_index(drop=True)

    english_common = stopwords.words('english')
    df = df[~df['word'].isin(english_common)]

    return df

In [65]:
game_df = clean_dataframe(game_before_df)
sword_df = clean_dataframe(swords_before_df)

sword_df.head(1500000)












,word
1,storm
3,swords
4,book
5,three
8,song
...,...
927892,nineteen
927893,wives
927894,previous
927895,table
